# First Steps Connecting to the Wereldculturen MS SQL Server

Using pyodbc to connect to the server and submit queries to the DB

In [7]:
import pandas as pd

In [1]:
import pyodbc 
server = 'tcp:azuredfserv.database.windows.net' 
database = 'Azuredf' 
username = 'Demouser' 
password = 'Knxdde#77' 
driver='{ODBC Driver 17 for SQL Server}'

# 0. Make Connection and List Databases on the Server

In [2]:
with pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT name, collation_name FROM sys.databases")
        row = cursor.fetchone()
        while row:
            print (str(row[0]) + " " + str(row[1]))
            row = cursor.fetchone()
            

master SQL_Latin1_General_CP1_CI_AS
azuredf SQL_Latin1_General_CP1_CI_AS


# 1. Get List of Tables in the Database

In [25]:
with pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("""select schema_name(t.schema_id) as schema_name,
                               t.name as table_name,
                               t.create_date,
                               t.modify_date
                               from sys.tables t""")
        
        
        for row in cursor.fetchall():
            print(row)
            print()

        
        cursor.execute("""select t.name
                               from sys.tables t""")

        
        table_names = cursor.fetchall()
        print(table_names)

('dbo', 'Person', datetime.datetime(2021, 2, 4, 8, 27, 28), datetime.datetime(2021, 2, 4, 8, 27, 28))

('dbo', 'ClassificationXRefs', datetime.datetime(2021, 2, 5, 11, 9, 55, 290000), datetime.datetime(2021, 2, 5, 11, 9, 55, 290000))

('dbo', 'Departments', datetime.datetime(2021, 2, 5, 11, 9, 55, 320000), datetime.datetime(2021, 2, 5, 11, 9, 55, 320000))

('dbo', 'Objects', datetime.datetime(2021, 2, 5, 11, 9, 55, 323000), datetime.datetime(2021, 2, 5, 11, 9, 55, 323000))

('dbo', 'Classifications', datetime.datetime(2021, 2, 5, 11, 9, 55, 330000), datetime.datetime(2021, 2, 5, 11, 9, 55, 330000))

[('Person', ), ('ClassificationXRefs', ), ('Departments', ), ('Objects', ), ('Classifications', )]


In [13]:
cursor.execute("""select * from sys.tables t""")

cursor.fetchall()

[('Person', 1525580473, None, 1, 0, 'U ', 'USER_TABLE', datetime.datetime(2021, 2, 4, 8, 27, 28), datetime.datetime(2021, 2, 4, 8, 27, 28), False, False, False, 0, None, 5, False, True, False, False, False, False, False, 0, False, False, 0, 'TABLE', False, False, 0, 'SCHEMA_AND_DATA', 0, 'NON_TEMPORAL_TABLE', None, False, False, None, None, None, False, False, -1, -1, 'INFINITE', 0, 'NON_LEDGER_TABLE', None),
 ('ClassificationXRefs', 1573580644, None, 1, 0, 'U ', 'USER_TABLE', datetime.datetime(2021, 2, 5, 11, 9, 55, 290000), datetime.datetime(2021, 2, 5, 11, 9, 55, 290000), False, False, False, 0, None, 8, False, True, False, False, False, False, False, 0, False, False, 0, 'TABLE', False, False, 0, 'SCHEMA_AND_DATA', 0, 'NON_TEMPORAL_TABLE', None, False, False, None, None, None, False, False, -1, -1, 'INFINITE', 0, 'NON_LEDGER_TABLE', None),
 ('Departments', 1589580701, None, 1, 0, 'U ', 'USER_TABLE', datetime.datetime(2021, 2, 5, 11, 9, 55, 320000), datetime.datetime(2021, 2, 5, 11, 

In [34]:
for tbl in table_names:
    n = tbl[0]
    cursor.execute(f"""SELECT COLUMN_NAME 
                        FROM INFORMATION_SCHEMA.COLUMNS
                        WHERE TABLE_NAME = '{n}' AND TABLE_SCHEMA='dbo'""")
    columns = cursor.fetchall()
            
            
    cursor.execute(f"select TOP 10 * from {n} ;")
    table = cursor.fetchall()
#            print(n, "has entries : ", len(cursor.fetchall()))
#     print(n)
#     print()
#     print(columns)
#     print()
#     print(table)
    
    
    df = pd.DataFrame.from_records(table, columns=[tup[0] for tup in columns])
   
    try:
        df["GSRowVersion"] = df["GSRowVersion"].str.decode("iso-8859-1")  # df.apply(lambda row: str.encode(row["GSRowVersion"]))
    except KeyError:
        pass
    
    print(n)
    print(df.head())
            
    print("\n", "-"*30, "\n\n")

Person
Empty DataFrame
Columns: [PersonId, FirstName, MiddelInitial, LastName, DateOfBirth]
Index: []

 ------------------------------ 


ClassificationXRefs
   ClassificationXRefID  ClassificationID      ID  TableID LoginID  \
0                     1               168   52305      108     DMU   
1                     2               168   52306      108     DMU   
2                     3               168   85800      108     DMU   
3                     4               168  185858      108     DMU   
4                     5               168  185859      108     DMU   

              EnteredDate  DisplayOrder GSRowVersion  
0 2013-03-14 14:31:58.473             1          ßªk  
1 2013-03-14 14:31:58.473             1          ß«  
2 2013-03-14 14:31:58.473             1          ß«q  
3 2013-03-14 14:31:58.473             1          ß«r  
4 2013-03-14 14:31:58.473             1          ß«s  

 ------------------------------ 


Departments
   DepartmentID                            